<h1 style="text-align:center; font-size:250%; font-family:Helvetica; color:#cc3399;"><b>Bakery Sales Data (EDA)</b></h1> 


<h3 style="text-align:center;  font-family:Helvetica;  "><b>Market Basket Analysis : Association Rule Mining</b> </h3> 


<img src=" https://img.freepik.com/free-vector/small-bread-store-interior-flat-vector-illustration-cartoon-woman-man-buying-snacks-shop-standing-line-business-food-bakery-store-concept_74855-10155.jpg?w=1380&t=st=1659065427~exp=1659066027~hmac=e66c0282d5a34c9a5e6c48e783f7e76a009bdc1d9a8efb88c3c62aca20fe3ed2" 
alt=" A Bakey Shope" style="width:60% ;height:600px; border-radius: 25px; margin-left: 20%">

<h3 style="text-align:left; font-family:Arial;"><b>1. Importing Necessary Dependencies</b> </h3> 

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder


<h3 style="text-align:left; font-family:Arial;"><b>2. Loading and Reading Dataset</b></h3> 

In [2]:
bakery_df = pd.read_csv("Bakery.csv")
bakery_df.head()

,TransactionNo,Items,DateTime,Daypart,DayType
0,1,Bread,2016-10-30 09:58:11,Morning,Weekend
1,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend
2,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend
3,3,Hot chocolate,2016-10-30 10:07:57,Morning,Weekend
4,3,Jam,2016-10-30 10:07:57,Morning,Weekend


In [3]:
print("Database dimension :", bakery_df.shape)
print("Dataset size       : ", bakery_df.size)

Database dimension : (20507, 5)
Dataset size       :  102535


In [4]:
bakery_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TransactionNo  20507 non-null  int64 
 1   Items          20507 non-null  object
 2   DateTime       20507 non-null  object
 3   Daypart        20507 non-null  object
 4   DayType        20507 non-null  object
dtypes: int64(1), object(4)
memory usage: 801.2+ KB


### Changing DateTime column from object to DateTime

In [5]:
bakery_df['DateTime'] = bakery_df['DateTime'].astype('datetime64[ns]')

In [6]:
bakery_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   TransactionNo  20507 non-null  int64         
 1   Items          20507 non-null  object        
 2   DateTime       20507 non-null  datetime64[ns]
 3   Daypart        20507 non-null  object        
 4   DayType        20507 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 801.2+ KB


In [7]:
bakery_df['TransactionNo'].nunique()

9465

In [8]:
bakery_df.describe(include=object)

,Items,Daypart,DayType
count,20507,20507,20507
unique,94,4,2
top,Coffee,Afternoon,Weekday
freq,5471,11569,12807


<h3 style="text-align:left; font-family:Arial;"><b>Data Summary:</b></h3>

<div style="
            display:fill;
            border-radius:5px;
            background-color: #f5d6eb;  
            font-size:100%;
            theme: cosmo;
            letter-spacing:0.5px">
<h3 style="padding-left: 20px; padding-top: 20px; color:#cc3399; font-family:Arial;"><b>Overview</b>
</h3>
<p style="padding-left: 20px; padding-right: 20px;font-size:110%;">
The dataset provides transaction details of all items purchased between 2016 and 2017 from the bakery online. The dataset has <b>20507</b> entries over <b>9000</b> transactions, and 4 columns.
</p>
<ul style="padding-left: 40px; padding-bottom: 20px;font-size:110%;">
    <li>Number of variables: 1</li>
    <li>Numeric variables: 1</li>
    <li>Categorical variables: 4</li>
    <li>Number of observations: 20507</li>
    <li>Total number of transactions: 9465</li>
    <li>Missing cells : 0</li>
</ul>
</div>

<div style="
            display:fill;
            border-radius:5px;
            background-color:#f5d6eb;  
            font-size:100%;
            theme:cosmo;
            letter-spacing:0.5px">
<h3 style="padding-left: 20px; padding-top: 20px; color:#cc3399; font-family:Arial;"><b>Variables</b>
</h3>
<ul style="padding-left: 40px; padding-bottom: 20px;  font-size:110%;">
    <li>TransactionNo : <b>9465</b> distinct values</li>
    <li><b>Items</b> has a high cardinality: <b>94</b> distinct values</li>
    <li><b>DateTime</b> has a high cardinality: <b>9182</b> distinct values</li>
    <li><b>Daypart</b> has <b>4</b> distinct values</li>
    <li><b>DayType</b> has <b>2</b> distinct values</li>
</ul>
</div>

<h3 style="text-align:left; font-family:Arial;"><b>3. Data Exploration and Visualization</b></h3> 

<h4 style="text-align:left; font-family:Arial;"><b>3.1 Let's look into the frequent items and the best sellers</b></h4> 

In [9]:
item_frequency = bakery_df['Items'].value_counts().sort_values(ascending=False)
item_frequency.head(10)

Coffee           5471
Bread            3325
Tea              1435
Cake             1025
Pastry            856
Sandwich          771
Medialuna         616
Hot chocolate     590
Cookies           540
Brownie           379
Name: Items, dtype: int64

In [65]:
from re import template

from matplotlib.pyplot import title
from pyparsing import White


fig = px.bar(item_frequency.head(10),
title="20 Most Frequent Items",
color=item_frequency.head(10),
text_auto=True,
width=1000,
height=600,
color_continuous_scale=px.colors.sequential.Magenta)

fig.update_layout(coloraxis_showscale = False, template="simple_white", margin=dict(t=60, b=0, l=100, r=0), titlefont=dict(size=25), title_x = 0.6)
fig.update_yaxes(showticklabels=False, title="")
fig.update_xaxes(title="")
fig.update_traces(textposition="outside", hovertemplate = '<b>%{x}</b><br>No. of Transactions: %{y}')

fig.show()

<p style=" font-size:110%;"> <b> Coffee is the best-selling product by far, followed by bread and tea. </b> </p>

<h4 style="text-align:left; font-family:Arial;"><b>3.2 Let's look into the peak hours of sales</b></h4> 

In [11]:
peak_hours = bakery_df.groupby('Daypart')['Items'].count().sort_values(ascending=False)
peak_hours

Daypart
Afternoon    11569
Morning       8404
Evening        520
Night           14
Name: Items, dtype: int64

In [64]:
from turtle import width
from matplotlib.pyplot import margins
from numpy import size


fig = go.Figure(data = [go.Pie(labels = ['Afternoon', ' Morning', 'Evening', 'Night'],
values=peak_hours,
title="Peak Selling Hours",
titlefont=dict(size=20),
textinfo='label+percent',
marker=dict(colors=px.colors.sequential.Magenta),
hole=0.5,)]
)

fig.update_layout(margin=dict(t=40, b=40, l=0, r=0), font=dict(size=18), showlegend=False)
fig.update_layout(width=900, height=600, template="simple_white")

fig.show()

<p style=" font-size:110%;"> <b> The bakery seems to be making most of its sales in the afternoon everyday with over 56% of the sales. Sales fall sharply after that. However the bakery makes a decent amount of sales in the morning as well. </b> </p>

<h4 style="text-align:left; font-family:Arial;"><b>3.3 Further let's look into the monthly and weekly sales</b></h4> 

<p style="font-size:110%;"> <b> Need to extract months and days from the dataset for further analysis. </b> </p>

In [13]:
from datetime import date


date_time = pd.to_datetime(bakery_df['DateTime'])
bakery_df['Day'] = date_time.dt.day_name()
bakery_df['Month'] = date_time.dt.month_name()
bakery_df['Year'] = "2016"          #date_time.dt.year()

bakery_df.head()

,TransactionNo,Items,DateTime,Daypart,DayType,Day,Month,Year
0,1,Bread,2016-10-30 09:58:11,Morning,Weekend,Sunday,October,2016
1,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend,Sunday,October,2016
2,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend,Sunday,October,2016
3,3,Hot chocolate,2016-10-30 10:07:57,Morning,Weekend,Sunday,October,2016
4,3,Jam,2016-10-30 10:07:57,Morning,Weekend,Sunday,October,2016


In [14]:
mpd = bakery_df.groupby('Day')['Items'].count().sort_values(ascending=False)
mpd

Day
Saturday     3554
Friday       3266
Sunday       3118
Monday       3035
Tuesday      2645
Thursday     2601
Wednesday    2288
Name: Items, dtype: int64

In [57]:
fig = px.bar(mpd, title="Most Productive Day", color=mpd, text_auto=True,color_continuous_scale=px.colors.sequential.Magenta)

fig.update_layout( coloraxis_showscale=False, template="simple_white", margin=dict(t=100, b=0, l=100, r=0), titlefont=dict(size=25), title_x = 0.4)
fig.update_yaxes(showticklabels=False, title="")
fig.update_xaxes(title="")
fig.update_traces(textposition='outside', hovertemplate = '<b>%{x}</b><br>No. of Transactions: %{y}')

fig.show()

<p style="font-size:110%;"> <b> For obvious reasons, the sales are high as expected during the weekends. However the sales seem to be quite uniform rest of the days. </b> </p>

In [16]:
mpm = bakery_df.groupby('Month')['Items'].count().sort_values(ascending=False)
mpm

Month
March        3220
November     3076
January      3027
February     2748
December     2647
April        1048
October      1041
May           924
July          741
June          739
August        700
September     596
Name: Items, dtype: int64

In [58]:
fig = px.bar(mpm, title="Most Productive Month", color=mpm, text_auto=True, color_continuous_scale=px.colors.sequential.Magenta)

fig.update_layout( coloraxis_showscale = False,template="simple_white", margin=dict(t=100, b=0, l=100, r=0), titlefont=dict(size=25), title_x = 0.4)
fig.update_traces(textposition='outside',  hovertemplate = '<b>%{x}</b><br>No. of Transactions: %{y}')
fig.update_yaxes(showticklabels=False, title=' ')
fig.update_xaxes(title=' ')


fig.show()

<p style=" font-size:110%;"> <b> The bakery seems to be heavily occupied and makes most of its business from November to March. </b> </p>

<div style="
            display:fill;
            border-radius:5px;
            background-color:#f5d6eb;  
            font-size:100%;
            theme:cosmo;
            letter-spacing:0.5px">
<h3 style="padding-left: 20px; padding-top: 20px; color: #cc3399; font-family:Arial;"><b>EDA Summary:</b></h3>
<p style="padding-left: 20px; padding-bottom: 20px; font-size:110%;"> 
<b>Coffee</b> is the best-selling product by far, followed by <b>bread</b> and <b>tea</b>. The bakery seems to be making most of its sales in the afternoon everyday with over <b>56%</b> of the sales. Sales fall sharply after that. However the bakery makes a decent amount of sales in the morning as well. For obvious reasons, the sales are high as expected during the weekends. However the sales seem to be quite uniform rest of the days. The bakery seems to be heavily occupied and makes most of its business from November to March.
</p>
</div>

<h3 style="text-align:left; font-family:Arial;"><b>4. Association Rules Generation</b></h3> 

<h4 style="text-align:left; font-family:Arial;"><b>4.1 Data Preparation for Association Rule Mining</b></h4>
<p style=" font-size:110%;"> <b> Apriori algorithm requires a dataframe with all the transactions one hot encoded for all the items. </b> </p>

- <h4 style="text-align:left; font-family:Arial;"><b>list of all the transactions</b></h4>

In [18]:
transaction = []
for item in bakery_df['TransactionNo'].unique():
    lst = list(set(bakery_df[bakery_df['TransactionNo'] == item]['Items']))
    transaction.append(lst)


transaction[:10]

[['Bread'],
 ['Scandinavian'],
 ['Cookies', 'Hot chocolate', 'Jam'],
 ['Muffin'],
 ['Pastry', 'Coffee', 'Bread'],
 ['Pastry', 'Medialuna', 'Muffin'],
 ['Pastry', 'Medialuna', 'Coffee', 'Tea'],
 ['Pastry', 'Bread'],
 ['Muffin', 'Bread'],
 ['Medialuna', 'Scandinavian']]

- <h4 style="text-align:left; font-family:Arial;"><b>one hot encoding</b></h4>

In [19]:
te = TransactionEncoder()
encoded_data = te.fit(transaction).transform(transaction)

data = pd.DataFrame(encoded_data, columns=te.columns_)
data.head()

,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


<h3 style="text-align:left; font-family:Arial;"><b>4.2 Association Rules Generation</b></h3>

- <h4 style="text-align:left; font-family:Arial;"><b>frequent items</b></h4>

In [20]:
frequent_items = apriori(data, use_colnames=True, min_support=0.02)
frequent_items.head()

,support,itemsets
0,0.036344,(Alfajores)
1,0.327205,(Bread)
2,0.040042,(Brownie)
3,0.103856,(Cake)
4,0.478394,(Coffee)


- <h4 style="text-align:left; font-family:Arial;"><b>association rules</b></h4>

In [21]:
rules = association_rules(frequent_items, metric="lift", min_threshold=1)

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Pastry),(Bread),0.086107,0.327205,0.029160,0.338650,1.034977,0.000985,1.017305
1,(Bread),(Pastry),0.327205,0.086107,0.029160,0.089119,1.034977,0.000985,1.003306
2,(Coffee),(Cake),0.478394,0.103856,0.054728,0.114399,1.101515,0.005044,1.011905
3,(Cake),(Coffee),0.103856,0.478394,0.054728,0.526958,1.101515,0.005044,1.102664
4,(Cake),(Tea),0.103856,0.142631,0.023772,0.228891,1.604781,0.008959,1.111865


<h2 style="text-align:left; font-family:Arial;"><b>5. Refining Rules</b></h3> 
<p style=" font-size:110%;"> <b> The confidence for a very frequent consequent is always high even if there is a very weak association. So this doesn't give a clearer picture. Here, coffee is by far the most frequent item and the best seller. It can therefore be recommended anyway with every other item. So, we can drop the rules recommending coffee to get a clearer picture of the real unknown rules generated from the data. </b> </p>

In [22]:
index_names = rules[rules['consequents'] == 'Coffee'].index
refinedRules = rules.drop(index_names).sort_values('lift', ascending=False)
refinedRules.drop(['leverage','conviction'], axis=1, inplace=True)
refinedRules = refinedRules.reset_index()
refinedRules

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,5,(Tea),(Cake),0.142631,0.103856,0.023772,0.166667,1.604781
1,4,(Cake),(Tea),0.103856,0.142631,0.023772,0.228891,1.604781
2,19,(Coffee),(Toast),0.478394,0.033597,0.023666,0.049470,1.472431
3,18,(Toast),(Coffee),0.033597,0.478394,0.023666,0.704403,1.472431
4,13,(Coffee),(Medialuna),0.478394,0.061807,0.035182,0.073542,1.189878
5,12,(Medialuna),(Coffee),0.061807,0.478394,0.035182,0.569231,1.189878
6,15,(Coffee),(Pastry),0.478394,0.086107,0.047544,0.099382,1.154168
7,14,(Pastry),(Coffee),0.086107,0.478394,0.047544,0.552147,1.154168
8,11,(Juice),(Coffee),0.038563,0.478394,0.020602,0.534247,1.116750
9,10,(Coffee),(Juice),0.478394,0.038563,0.020602,0.043065,1.116750


<h2 style="text-align:left; font-family:Arial;"><b>Summary:</b></h2> 

<div style="
            display:fill;
            border-radius:5px;
            background-color:#f5d6eb;  
            font-size:100%;
            theme:cosmo;
            letter-spacing:0.5px">
<h3 style="padding-left: 20px; padding-top: 20px; color:  #cc3399; font-family:Arial;"><b>Insights</b>
</h3>
<ul style="padding-left: 40px; padding-bottom: 20px;  font-size:110%;">
<li>Coffee is the bestseller of this bakery and it shows association with <b>8</b> other items.</li>
<li>Over <b>11%</b> coffee lovers also buy cake along with while almost <b>10%</b> of them buy pastry along with it.</li>
<li>Over <b>16%</b> of tea consumers also buy cakes and over <b>22%</b> cake lovers also buy tea.</li>
<li>Among the pastry lovers, over <b>33%</b> of them also buy bread, while nearly <b>9%</b> of those who buy pastry also buy bread.</li>
</ul>
</div>

<div style="
            display:fill;
            border-radius:5px;
            background-color:#f5d6eb;  
            font-size:100%;
            theme:cosmo;
            letter-spacing:0.5px">
<h3 style="padding-left: 20px; padding-top: 20px; color:#cc3399; font-family:Arial;"><b>Business Strategy</b>
</h3>
<p style="padding-left: 20px;  font-size:110%;">
There are a couple of strategies that the bakery can adopt if is yet to use, to increase its sales considering the associations we have seen between coffee and its 8 partners.
</p>
<ul style="padding-left: 40px; padding-bottom: 20px; font-size:110%;">
<li>Promotional discounts on these items can entice customers to buy coffee or the other way round.</li>
<li>Arranging placements of these items close to coffee ordering counter can be a good strategy to tempt customers into buying them.</li>
</ul>
</div>